In [28]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler
from utils import neg_sample

import json
import math
import os
import random
from scipy.sparse import csr_matrix
from torch.utils.data import Dataset

from datasets import PretrainDataset
from models import S3RecModel
from trainers import PretrainTrainer
from utils import (
    EarlyStopping,
    check_path,
    get_item2attribute_json,
    get_user_seqs_long,
    set_seed,
)

In [5]:
# args.data_file = args.data_dir + args.data_name + '.txt'
data_file = "/opt/ml/input/data/train/train_ratings.csv"
item2attribute_file = "/opt/ml/input/data/train/Ml_item2attributes.json"
# concat all user_seq get a long sequence, from which sample neg segment for SP
user_seq, max_item, long_sequence = get_user_seqs_long(data_file)
item2attribute, attribute_size = get_item2attribute_json(item2attribute_file)

In [26]:
item_size = max_item + 2
mask_id = max_item + 1
attribute_size = attribute_size + 1

In [33]:
len(user_seq)

31360

In [34]:
31360*50

1568000

In [29]:
part_sequence = []
for seq in user_seq:
    input_ids = seq[-(50 + 2) : -2]  # keeping same as train set
    for i in range(len(input_ids)):
        part_sequence.append(input_ids[: i + 1])

In [35]:
part_sequence

[[5225],
 [5225, 1046],
 [5225, 1046, 64034],
 [5225, 1046, 64034, 53996],
 [5225, 1046, 64034, 53996, 43932],
 [5225, 1046, 64034, 53996, 43932, 26887],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319, 51698],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319, 51698, 5444],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319, 51698, 5444, 6731],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455,
  56145],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455,
  56145,
  968],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455,
  56145,
  968,
  54995],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  

In [37]:
writers_df = pd.read_csv("./data/train/writers.tsv", sep="\t")
array, index = pd.factorize(writers_df["writer"])
writers_df["writer"] = array
wirters_list = writers_df.groupby("item")["writer"].apply(list)

In [38]:
wirters_list

item
1         [296, 320, 748, 891, 2090, 2351, 2623]
2                             [1304, 2445, 2521]
3                                         [1435]
4                                          [466]
5                       [1165, 1226, 1812, 2313]
                           ...                  
118696              [183, 579, 2484, 2491, 2570]
118900                                    [1353]
118997                                    [1576]
119141                              [2162, 2895]
119145                  [2528, 2727, 2903, 2932]
Name: writer, Length: 5648, dtype: object